In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline
pd.set_option("display.max_columns",999)

In [2]:
df = pd.read_pickle("df_outdel.pkl")
df_FE = pd.read_pickle('df_FE4.pkl')

In [3]:
df.shape

(2896, 80)

In [4]:
df_FE.shape

(2896, 41)

## Encoding for modeling

https://contrib.scikit-learn.org/categorical-encoding/index.html#

In [5]:
numeric_col=[]
categoric_col=[]
tempdf=pd.DataFrame(df_FE.dtypes, columns=['type'])
for i in range(len(df_FE.columns)) :
    if tempdf['type'][i] == 'object' :
        categoric_col.append(tempdf.index[i])
    else :
        numeric_col.append(tempdf.index[i])

In [6]:
from category_encoders.cat_boost import CatBoostEncoder

In [7]:
df.SalePrice.isnull().sum()

1459

In [8]:
len(df)-df.SalePrice.isnull().sum()

1437

In [9]:
ce = CatBoostEncoder(cols=categoric_col)

ce.fit(X=df_FE[:1437], y=np.log(df['SalePrice'][:1437]), use_cat_names=True)
temp = ce.transform(df_FE)

## Linear model

In [10]:
X_train = temp[:1437]
y_train = np.log(df['SalePrice'][:1437])
X_test = temp[1437:]

In [12]:
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso

In [13]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [15]:
preprocess = FeatureUnion([('scaler',StandardScaler())])

pipe = Pipeline([('preprocess', preprocess),
                 ('model', Ridge())])

search = [{'model__alpha' : [0.01, 0.1, 1, 5]}]

ridge_grid = GridSearchCV(pipe, search, cv=5, scoring=make_scorer(rmse, greater_is_better=False), 
                          verbose=1, n_jobs=-1)

ridge_grid.fit(X_train, y_train)

print(f'best parameter : {ridge_grid.best_params_}')
print(f'best score : {ridge_grid.best_score_}')

ridge_model = ridge_grid.best_estimator_
pred = ridge_model.predict(X_test)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


best parameter : {'model__alpha': 5}
best score : -0.12199964074681544


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    2.7s finished


In [18]:
sub = pd.read_csv('data/sample_submission.csv')

In [20]:
sub['SalePrice'] = np.exp(pred)

In [22]:
sub.to_csv('sub.csv',index=False)